In [1]:
#Necessary Libraries
import numpy as np
import tensorflow
from keras.utils import np_utils
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, ResNet50, Xception

In [2]:
#Mounting the drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Unzipping the nature_12K.zip dataset
zip_path = "/content/drive/MyDrive/Dataset /nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [4]:
#Installing the wandb library

!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 181 kB 45.5 MB/s 
     |████████████████████████████████| 144 kB 47.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [5]:
# Storing the training and testing directories
import os 
Tags = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
no_class = 10
train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'

In [16]:
# Generating the dataset for training and validation
def train_val__test_data_generation(image_size=128,batch_size=256,augment_data=False):
    #Augmenting the data to avoid overfitting
    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=45,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=False)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    #Generating data batch by batch in order to make the model faster to run the dataset
    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size, subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size, subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size,image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [ ]:
def PreTrainedModel():
    #default configurations
    config_defaults = {
        'weight_decay': 0.005,
        'dropout': 0.2,
        'batch_size': 64,
        'epochs': 10,
        'batch_norm': True,
        'augment_data': True,
        'dense_size': 256,
        'seed': 1234,
        'pre_trained_model': 'Xception'
    }
    image_size=128
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    #config will store the hyperparameters
    config = wandb.config
    wandb.run.name = 'pmodel_'+ str(config.pre_trained_model)+'dense_size_'+ str(config.dense_size)+'_dropout_'+str(config.dropout)+'_bn_'+str(config.batch_norm)+'_ag_'+ str(config.augment_data)

    if config.pre_trained_model == 'Inceptionv3':
        image_size=299
        pmodel = InceptionV3(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Inceptionresnetv2':
        image_size=299
        pmodel = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Xception':
        image_size=299
        pmodel = Xception(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Resnet50':
        image_size=224
        pmodel = ResNet50(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
  

    pmodel.trainable = False
    model = Sequential()
    model.add(pmodel)
    model.add(Flatten())
    if config.batch_norm:
        model.add(BatchNormalization())
    model.add(Dense(config.dense_size, activation='relu'))
    model.add(Dropout(config.dropout))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    train_generator, val_generator, test_generator = train_val__test_data_generation(image_size=image_size,batch_size=config.batch_size,augment_data=config.augment_data)
    hist=model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])
    val_acc=max(hist.history['val_accuracy'])
    params={'batch_norm':config.batch_norm,'augmentation':config.augment_data,'dropout':config.dropout,
            'pre_trained_model':config.pre_trained_model,'val_acc':val_acc}
    wandb.log(params)

In [12]:
#Configurations to find out the best hyperparameters out of them
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'pre_trained_model': {
            'values':['Inceptionv3','Inceptionresnetv2','Xception','Resnet50']
        },
        'dropout': {
            'values': [0, 0.2, 0.3, 0.4]
        },
        'batch_norm':{
            'values': [True,False]
        },
        'augment_data': {
            'values': [True,False]
        },
        'batch_size': {
            'values': [32, 64, 128, 256]
        },
        'dense_size':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [13]:
def PreTrainedModel():
    #default configurations
    config_defaults = {
        'weight_decay': 0.005,
        'dropout': 0.2,
        'batch_size': 64,
        'epochs': 10,
        'batch_norm': True,
        'augment_data': True,
        'dense_size': 256,
        'seed': 1234,
        'pre_trained_model': 'Xception'
    }
    image_size=128
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    #config will store the hyperparameters
    config = wandb.config
    wandb.run.name = 'pmodel_'+ str(config.pre_trained_model)+'dense_size_'+ str(config.dense_size)+'_dropout_'+str(config.dropout)+'_bn_'+str(config.batch_norm)+'_ag_'+ str(config.augment_data)

    if config.pre_trained_model == 'Inceptionv3':
        image_size=299
        pmodel = InceptionV3(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Inceptionresnetv2':
        image_size=299
        pmodel = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Xception':
        image_size=299
        pmodel = Xception(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Resnet50':
        image_size=224
        pmodel = ResNet50(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
  

    pmodel.trainable = False
    model = Sequential()
    model.add(pmodel)
    model.add(Flatten())
    if config.batch_norm:
        model.add(BatchNormalization())
    model.add(Dense(config.dense_size, activation='relu'))
    model.add(Dropout(config.dropout))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    train_generator, val_generator, test_generator = train_val__test_data_generation(image_size=image_size,batch_size=config.batch_size,augment_data=config.augment_data)
    hist=model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])
    val_acc=max(hist.history['val_accuracy'])
    params={'batch_norm':config.batch_norm,'augmentation':config.augment_data,'dropout':config.dropout,
            'pre_trained_model':config.pre_trained_model,'val_acc':val_acc}
    wandb.log(params)

In [17]:
sweep_id = wandb.sweep(sweep_config, project="CS6910_DL_Assignment2", entity="nomads")


Create sweep with ID: i1haqmcw
Sweep URL: https://wandb.ai/nomads/CS6910_DL_Assignment2/sweeps/i1haqmcw


In [18]:
wandb.agent('i1haqmcw',PreTrainedModel, count = 1)

wandb: Agent Starting Run: q4udgnxe with config:
wandb: 	augment_data: False
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	dense_size: 256
wandb: 	dropout: 0.3
wandb: 	pre_trained_model: Inceptionv3
wandb: 	weight_decay: 0


87924736/87910968 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
141/141 [==============================] - 192s 1s/step - loss: 12.7439 - accuracy: 0.6870 - val_loss: 4.5944 - val_accuracy: 0.7227 - _timestamp: 1648701636.0000 - _runtime: 206.0000
Epoch 2/10
141/141 [==============================] - 157s 1s/step - loss: 2.5962 - accuracy: 0.8621 - val_loss: 4.8127 - val_accuracy: 0.7618 - _timestamp: 1648701794.0000 - _runtime: 364.0000
Epoch 3/10
141/141 [==============================] - 152s 1s/step - loss: 1.1518 - accuracy: 0.9109 - val_loss: 5.3205 - val_accuracy: 0.7648 - _timestamp: 1648701947.0000 - _runtime: 517.0000
Epoch 4/10
141/141 [==============================] - 155s 1s/step - loss: 0.7125 - accuracy: 0.9308 - val_loss: 5.1628 - val_accuracy: 0.7768 - _timestamp: 1648702102.0000 - _runtime: 672.0000
Epoch 5/10
141/141 [==========

accuracy,▁▆▇▇██████
augmentation,▁
batch_norm,▁
dropout,▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▁▁▁▁▁▁▁▁
val_acc,▁
val_accuracy,▁▅▅▆█▅▅█▆▅
val_loss,▁▂▄▃▄▃▆▆▆█
accuracy,0.94911
augmentation,False
